In this section I use API call to query fundamental data from EOD database. Periously I spent a lot of time to replicate [Stefan Jansen's practice to query fundamental data from government resources](https://github.com/stefan-jansen/machine-learning-for-trading/blob/main/02_market_and_fundamental_data/04_sec_edgar/edgar_xbrl.ipynb). This attemp finally failed because the government resources collect almost every aspect of the corporation (even those data only appear once during their whole life time or those features only one ticker has can all be included), thus forming a too sparse feature matrix with 10,000 features to handle during intermediate data transformation.

In [1]:
api_key = "633ca2a0d16e21.68445472"


# Libraries
import pandas as pd
from eod import EodHistoricalData
from functools import reduce
from datetime import datetime, timedelta
# Importing and assigning the api key
    
# EOD Historical Data client
client = EodHistoricalData(api_key)



In [3]:
fund_data = client.get_fundamental_equity('AAPL')

In [7]:
fund_data.keys()

dict_keys(['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals', 'SplitsDividends', 'AnalystRatings', 'Holders', 'InsiderTransactions', 'ESGScores', 'outstandingShares', 'Earnings', 'Financials'])

In [26]:
fund_data['SharesStats']

{'SharesOutstanding': 16070800384,
 'SharesFloat': 16053877710,
 'PercentInsiders': 0.07100000000000001,
 'PercentInstitutions': 59.582,
 'SharesShort': None,
 'SharesShortPriorMonth': None,
 'ShortRatio': None,
 'ShortPercentOutstanding': None,
 'ShortPercentFloat': None}

In [20]:
pd.DataFrame(fund_data['Earnings'])

,History,Trend,Annual
2023-06-30,"{'reportDate': '2023-07-26', 'date': '2023-06-...",NaN,NaN
2023-03-31,"{'reportDate': '2023-04-26', 'date': '2023-03-...",NaN,NaN
2022-12-31,"{'reportDate': '2023-01-25', 'date': '2022-12-...","{'date': '2022-12-31', 'period': '+1q', 'growt...",NaN
2022-09-30,"{'reportDate': '2022-10-26', 'date': '2022-09-...","{'date': '2022-09-30', 'period': '0y', 'growth...","{'date': '2022-09-30', 'epsActual': 4.82}"
2022-06-30,"{'reportDate': '2022-07-28', 'date': '2022-06-...","{'date': '2022-06-30', 'period': '0q', 'growth...",NaN
...,...,...,...
1994-09-30,"{'reportDate': '1994-10-17', 'date': '1994-09-...",NaN,"{'date': '1994-09-30', 'epsActual': 0.0539}"
1994-06-30,"{'reportDate': '1994-06-30', 'date': '1994-06-...",NaN,NaN
1994-03-31,"{'reportDate': '1994-03-31', 'date': '1994-03-...",NaN,NaN
1993-12-31,"{'reportDate': '1993-12-31', 'date': '1993-12-...",NaN,NaN


In [2]:
def getFundamentals(ticker):
    """
    Returns the fundamental data from the financial data API.  Combines the quarterly balance 
    sheet, cash flow, income statement, and earnings for a specific stock ticker.
    """
    
    # Getting data
    fund_data = client.get_fundamental_equity(ticker)
    
    # Financials
    bal = pd.DataFrame(fund_data['Financials']['Balance_Sheet']['quarterly']).T
    
    cf = pd.DataFrame(fund_data['Financials']['Cash_Flow']['quarterly']).T
    
    inc = pd.DataFrame(fund_data['Financials']['Income_Statement']['quarterly']).T
    
    # Earnings
    earn = pd.DataFrame(fund_data['Earnings']['History']).T
    
    # Merging them together
    df = reduce(
        lambda left,right: pd.merge(
            left,
            right,
            left_index=True, 
            right_index=True, 
            how='outer',
            suffixes=('', '_drop')
        ), 
        [bal, cf, inc, earn]
    )
    
    # Dropping redundant date and duplicate columns
    dup_cols = [i for i in df.columns if "date" in i or "Date" in i or "_drop" in i]
    
    df = df.drop(dup_cols, axis=1)
    
    df["Ticker"] = ticker
    
    return df

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]

In [4]:
import time
data = pd.DataFrame([])
for ticker in df['Symbol'].tolist():
    try:
        d = getFundamentals(ticker)
        data = pd.concat([data, d])
        print("Collected " + ticker)
        time.sleep(1)
    except:
        print("Ignore " + ticker)

Collected MMM
Collected AOS
Collected ABT
Collected ABBV
Collected ABMD
Collected ACN
Collected ATVI
Collected ADM
Collected ADBE
Collected ADP
Collected AAP
Collected AES
Collected AFL
Collected A
Collected APD
Collected AKAM
Collected ALK
Collected ALB
Collected ARE
Collected ALGN
Collected ALLE
Collected LNT
Collected ALL
Collected GOOGL
Collected GOOG
Collected MO
Collected AMZN
Collected AMCR
Collected AMD
Collected AEE
Collected AAL
Collected AEP
Collected AXP
Collected AIG
Collected AMT
Collected AWK
Collected AMP
Collected ABC
Collected AME
Collected AMGN
Collected APH
Collected ADI
Collected ANSS
Collected AON
Collected APA
Collected AAPL
Collected AMAT
Collected APTV
Collected ANET
Collected AJG
Collected AIZ
Collected T
Collected ATO
Collected ADSK
Collected AZO
Collected AVB
Collected AVY
Collected BKR
Collected BALL
Collected BAC
Collected BBWI
Collected BAX
Collected BDX
Collected WRB
Collected BRK.B
Collected BBY
Collected BIO
Collected TECH
Collected BIIB
Collected BLK


In [5]:
data

,currency_symbol,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,otherCurrentLiab,commonStock,...,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,preferredStockAndOtherAdjustments,beforeAfterMarket,currency,epsActual,epsEstimate,epsDifference,surprisePercent,Ticker
1985-09-30,USD,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1985-12-31,USD,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1986-03-31,USD,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1986-06-30,USD,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,MMM
1986-09-30,USD,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,USD,13770000000.00,1390000000.00,None,507000000.00,9190000000.00,4580000000.00,274000000.00,1269000000.00,5000000.00,...,529000000.00,529000000.00,None,BeforeMarket,USD,1.2,1.22,-0.02,-1.6393,ZTS
2022-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BeforeMarket,USD,None,1.25,None,None,ZTS
2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AfterMarket,USD,None,None,None,None,ZTS
2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,BeforeMarket,USD,None,None,None,None,ZTS


In [6]:
data.to_csv("fundamental.csv")